# Model 2

Second attempt at modelling the Iron Hypothesis with graphs. Here we use separate nodes for Increase/Decrease events and their variables.

In [1]:
from py2neo import Graph, Node, Relationship
from scripts.vis import draw

Build initial graph.

In [2]:
graph = Graph()

graph.delete_all()
options = {"Var": "name", "Increase": "label", "Decrease": "label"}

event1 = Node("Increase", label="+")
iron = Node("Var", name="iron to seawater")
graph.create(Relationship(event1, "THEME", iron))

event2 = Node("Increase", label="+")
phytoplankton = Node("Var", name="phytoplankton")
graph.create(Relationship(event2, "THEME", phytoplankton))

graph.create(Relationship(event1, "CAUSES", event2))

producer = Node("Var", name="primary producer")
graph.create(Relationship(phytoplankton, "ISA", producer))

event3 = Node("Increase", label="+")
graph.create(Relationship(event3, "THEME", producer))

event4 = Node("Increase", label="+")
productivity = Node("Var", name="productivity")
graph.create(Relationship(event4, "THEME", productivity))

graph.create(Relationship(event3, "CAUSES", event4))


draw(graph, options)

Inference step:
if phytoplankton is increasing and phytoplankton is a primary producer, 
then primary producers are increasing.

In [3]:
query = """
MATCH (event1:Increase) -[:THEME]-> (var1) -[:ISA]-> (var2) <-[:THEME]- (event2:Increase)
CREATE (event1) -[:IMPLIES]-> (event2)
"""

graph.cypher.execute(query)

draw(graph, options)

Build second part of graph

In [4]:
net_high_lat_prod = Node("Var", name="the net high latitude productivity")
event5 = Node("Increase", label="+")
graph.create(Relationship(event5, "THEME", net_high_lat_prod))

pco2_ice_age = Node("Var", name="PCO2 of the last ice age")
event6 = Node("Decrease", label="-")
graph.create(Relationship(event6, "THEME", pco2_ice_age))

graph.create(Relationship(event5, "CAUSES", event6))

graph.create(Relationship(productivity, "GENERALISES", net_high_lat_prod))

pco2 = Node("Var", name="PCO2")
graph.create(Relationship(pco2, "GENERALISES", pco2_ice_age))

draw(graph, options)

Second inference step:

an increase in the net high lattitude productivity implies an increase in productivity 

In [5]:
query = """
MATCH (event1:Increase) -[:THEME]-> (var1) -[:GENERALISES]-> (var2) <-[:THEME]- (event2:Increase)
CREATE (event2) -[:IMPLIES]-> (event1)
"""

graph.cypher.execute(query)

draw(graph, options)

Second inference: 

a decrease in PCO2 in the last ice age implies a decrease in PCO2.

In [6]:
query = """
MATCH (event1:Decrease) -[:THEME]-> (var1) <-[:GENERALISES]-> (var2) 
CREATE (event1) -[:IMPLIES]-> (:Decrease {label: "-"}) -[:THEME]-> (var2)
"""

graph.cypher.execute(query)

draw(graph, options)

Now find variables related to "iron to seawater" by following paths over CAUSES, IMPLIES or THEME relations.

In [66]:
query = """
MATCH (var1:Var {name:'iron to seawater'}) -[:CAUSES|IMPLIES|THEME*]- (var2:Var) 
RETURN var1.name, var2.name
"""

graph.cypher.execute(query)

   | var1.name        | var2.name                         
---+------------------+------------------------------------
 1 | iron to seawater | phytoplankton                     
 2 | iron to seawater | primary producer                  
 3 | iron to seawater | productivity                      
 4 | iron to seawater | the net high latitude productivity
 5 | iron to seawater | PCO2 of the last ice age          
 6 | iron to seawater | PCO2                              

Explicitly match the events, so we know if a variable if changing, increasing or decreasing.

In [7]:
query = """
MATCH (var1:Var {name:'iron to seawater'}) <-[:THEME]- (event1) -[:CAUSES|IMPLIES|THEME*]- (event2) -[:THEME] -> (var2:Var) 
RETURN event1.label, var1.name, event2.label, var2.name
"""

graph.cypher.execute(query)

   | event1.label | var1.name        | event2.label | var2.name                         
---+--------------+------------------+--------------+------------------------------------
 1 | +            | iron to seawater | +            | phytoplankton                     
 2 | +            | iron to seawater | +            | primary producer                  
 3 | +            | iron to seawater | +            | productivity                      
 4 | +            | iron to seawater | +            | the net high latitude productivity
 5 | +            | iron to seawater | -            | PCO2 of the last ice age          
 6 | +            | iron to seawater | -            | PCO2                              